本文将对 Rust 中的 Rc 进行分析，来理解 如下的概念：
- Rc 的内存布局
- Rc 的实现原理
- Rust 是如何处理所有权和引用关系的

使用 Rust 进行开发的时候，一般的，每个数据（或者称为对象、结构体）有且只有一个所有者，当所有者被释放的时候，数据也会被释放，从而实现了内存的自动管理而不至于出现内存泄漏。在其他地方需要访问这个数据的时候，我们可以通过引用来访问，通过从 owner 获得 &T 或者 &mut T 来访问数据。 Rust 有一套所有权和引用的规则，保证了数据的安全性。这些规则包括：
- 一个数据只能有一个所有者
- 引用的生命周期不能超过所有者的生命周期
- 引用的可变性不能超过所有者的可变性
- 当存在可变引用的时候，不允许有其他引用（类似于数据库的写操作需要一个互斥锁，而读操作则是共享锁）

这套规则保证了数据的安全性，但是也带来了限制，你不能自由的访问数据：包括：
- 数据生命周期很复杂，难以确定一个owner。
- 数据需要在多个地方访问，包括读、写操作。这些操作具有独立性，无法预先确定。

在这种情况下，你如果要和 所有权、只读借用、读写借用的规则进行斗争的话，会是一个非常困难且艰巨的任务，甚至在某些情况下，你可能会发现这是不可能的（生命周期的不确定性、读写路径的不确定性）。而这个时候，我们可以对比 python/java 等语言，这些语言提供了垃圾回收机制，可以让我们不用关心内存的释放，对象的引用可以随意的传递，可以在任何位置自由的读写对象的内容，这种方式是极度自由的，但是也会带来另外的一些问题：
1. 依赖 GC 来释放内存，GC 会带来一定的性能损耗，以及对内存的使用效率降低（JVM 中的heap大部分时间都是垃圾，从而需要更多的运行内存）
2. 读写操作可能会带来数据竞争，包括多线程安全的问题，或者Java中常见的一类异常：ConcurrentModificationException

对比 Rust 和 Java 的内存管理，可以看到两者分别走了一个极端：
- Rust 是极端的管控，涉及到数据的所有权，可读借用，读写借用以及其有效期的管控。这种极端的管控，确保了安全、高效的内存管理，但是失去了自由。
- Java 是极端的自由，GC 管理内存，对象的引用可以随意传递，可以在任何位置自由的读写对象的内容。极端的自由，带来了性能损耗和数据竞争的问题，把这些问题交给了开发者来解决。

所幸的是，Rust 可以通过库的方式，提供了 Rc 来解决这个问题，Rc 是一个类似于 Java 引用的概念，我们也可以象Java一样，在 heap 中分配一个对象，然后通过 Rc 来引用这个对象，并且可以任意的进行传递（暂不考虑多线程的问题，在多线程下，要使用Rc的一个变体 Arc），相当于每个 Rc 都获得了对象的一定的所有权，只有当所有的 Rc 都被释放的时候，对象才会被释放。这种方式，可以让我们在 Rust 中，也可以实现类似于 Java 的自由度。

在这篇文章中，我们将对 Rc 进行分析，来理解 Rc 的内存布局、实现原理，以及 Rust 是如何处理所有权和引用关系的。同时，我们也可以理解强引用和弱引用的概念，以及如何在 Rust 中实现弱引用。

## 语言级的所有权、引用

## Library级的所有权、引用


In [11]:
use std::rc::Rc;

#[derive(Debug)]
struct User {
    name: String,
    age: u32,
}

let user1 = Rc::new(User {
    name: String::from("Alice"),
    age: 30
});

println!("name = {}", user1.name);  // auto deref

println!("ref count = {}", Rc::strong_count(&user1));  // ref count = 1
println!("weak ref count = {}", Rc::weak_count(&user1));  // weak ref count = 0

user1

name = Alice
ref count = 1
weak ref count = 0


User { name: "Alice", age: 30 }

## Rc 的 借用

Rc<T> 实现了 Borrow<T> 和 Deref<T>，所以 Rc<T> 可以被当做 &T 来使用，这样就可以实现 Rc<T> 的借用。 
```rust
trait Borrow<T: ?Sized> {
    fn borrow(&self) -> &T;
}
trait Deref {
    type Target: ?Sized;
    fn deref(&self) -> &Self::Target;
}
```
这两个接口几乎是一样的，都是返回一个引用，不同的是 Borrow 返回的是 &T，而 Deref 返回的是 &Self::Target。 不同之处：
1. Borrow<T> 使用了范型约束，理论上，某个类型可以实现多个 Borrow<T1> 和 Borrow<T2>。 
2. Deref 使用了关联类型，任何实现 Deref 的类型，都需要定义一个（且只能定义一个）关联类型 Target，这个类型就是 deref 方法返回的类型。

Rc<T> 可以理解为一个共享所有权的只能指针，因为其共享所有权，所以可以获得 T 的借用（但其借用的生命周期不得超过当前 Rc 的生命周期）。由于Rc 是共享所有权，因此，无法获得可写借用（否则，通过其他共享所有权的 Rc 也可能同时获得只读借用或者读写借用，在这种情况下，就会出现同时出现多个读写借用，或者读写借用 与 只读借用同时出现的情况，这会违背数据的安全性）。

In [17]:
use std::rc::Rc;
use std::borrow::Borrow;
use std::ops::Deref;

fn test() {
    let v1 = Rc::new(10);
    let v2 = Rc::clone(&v1);

    let v21: &i32 = &v2;     // same as (&v2).deref()
    let v22: &i32 = v2.deref();   // ok 
    // let v22: &i32 = v2;  // error: expected `&i32`, found struct `std::rc::Rc`
    
    let v23: &i32 = v2.borrow();    // same as v2.deref()
    let v24: &i32 = v2.as_ref();    // same as v2.deref()
    
    let v25: &i32 = &**v2;          // same as v2.deref().deref()


    // drop(v2);
    println!("v2ref = {}", v21);
}

test();

v2ref = 10


## 自动解引用
自动解引用这个语法糖让代码更为简洁，但也会为阅读代码带来一些挑战。

自动deref的规则是，如果类型T可以解引用为U，即T: Deref<U>，则 &T 可以转为 &U 

在 Zig 这样的语言中，强调 [No hidden control flow](https://ziglang.org/learn/why_zig_rust_d_cpp/#no-hidden-control-flow) :
> If Zig code doesn’t look like it’s jumping away to call a function, then it isn’t. This means you can be sure that the following code calls only foo() and then bar(), and this is guaranteed without needing to know the types of anything:

>```zig 
> var a = b + c.d;
> foo();
> bar();
>```

引入了 自动解引用后，代码的执行流程就不再那么明显了（隐式）。这也是语法糖在带来便利（可读）的同时，在可理解性上的一些损失。